In [1]:
import skimage
import os
import pydicom
from matplotlib import pyplot as plt
import skimage.feature
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

dirname = "Data/Reg" #npy files dir name

In [2]:
filenames = [el for el in os.listdir(dirname) if el.endswith('.npy')]
dataset = np.zeros((len(filenames), 486), dtype=np.float32)
for i, filename in enumerate(filenames):
    img = np.load(os.path.join(dirname, filename))[:,:, 0]
    sobel_img = skimage.filters.sobel(img)
    if sobel_img.shape[0] == 432:
        hog, hog_image = skimage.feature.hog(skimage.transform.rescale(sobel_img, 1/11), orientations=9, visualize=True)
    else:
        hog, hog_image = skimage.feature.hog(skimage.transform.rescale(sobel_img, 1/10), orientations=9, visualize=True)
    dataset[i] = hog

/Users/Gabriel/Development/anaconda3/envs/WebValley/lib/python3.7/site-packages/skimage/transform/_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


# PCA

In [3]:
pca = PCA(n_components=26)
pca.fit(dataset)

PCA(copy=True, iterated_power='auto', n_components=26, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [4]:
print(pca.explained_variance_ratio_)

[0.16251983 0.06933736 0.04642825 0.04330009 0.03451309 0.02816769
 0.02274754 0.02145284 0.01891205 0.01802524 0.01760705 0.0161579
 0.01474922 0.0138591  0.013033   0.01244135 0.01222422 0.01158413
 0.01086377 0.0104156  0.01009683 0.00973011 0.00923867 0.00875507
 0.00869902 0.00850614]


In [5]:
pca_dataset = pca.transform(dataset)

# KMeans Clustering

In [6]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(pca_dataset)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [7]:
kmeans.labels_

array([1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,

In [8]:
import _pickle as pickle
pickle.dump(kmeans, open("model.pkl", 'wb'))

In [ ]:
for i, filename in enumerate(filenames):
    if not "rev" in filename:
        print("Label: ", kmeans.labels_[i])
        img = np.load(dirname + "/" + filename)
        ogfile = pydicom.read_file("Data/Raw/" + filename[:][:-4] + ".dcm")
        plt.imshow(ogfile.pixel_array)
        plt.show()
        plt.imshow(img)
        plt.show()

# Move and Load Files

In [10]:
for i, filename in enumerate(filenames):
    os.rename(os.path.join(dirname, filename), os.path.join("Data", str(kmeans.labels_[i]), filename))

In [ ]:
for filename in os.listdir("Data/1"):
    if not "rev" in filename:
        img = np.load("Data/1/" + filename)
        ogfile = pydicom.read_file("Data/Raw/" + filename[:][:-4] + ".dcm")
        plt.imshow(ogfile.pixel_array)
        plt.show()

In [ ]:
img = np.load(dirname + "/" + "422.npy")
ogfile = pd.read_file("Data/Raw/" + "422.dcm")
plt.imshow(ogfile.pixel_array)
plt.show()
plt.imshow(img)
plt.show()